# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Export cities csv used in WeatherPy exercise and create Data frame. Display first 5 rows
output_data_file = "cities.csv"
city_data_df = pd.read_csv(output_data_file)
city_data_df.head()



,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Cape Town,20,ZA,1584285001,60,-33.93,18.42,71.01,31.09
1,1,Juegang,70,CN,1584285010,78,32.32,121.19,47.41,16.26
2,2,Ushuaia,75,AR,1584285011,67,-54.80,-68.30,57.20,4.47
3,3,Isangel,100,VU,1584284836,82,-19.55,169.27,82.71,29.84
4,4,Mataura,63,NZ,1584284838,94,-46.19,168.86,53.01,1.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps with api key
gmaps.configure(api_key=g_key)

#Define locations as Lat and Lng
locations = city_data_df[["Lat", "Lng"]].astype(float)

#Define weight as Humidity
weight = city_data_df["Humidity"].astype(float)


#Add heatmap layer to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weight, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Define weather conditions for vacation destination
vacay_df = city_data_df
vacay_df = vacay_df[(vacay_df["Max Temp"] > 72) & (vacay_df["Max Temp"] <=83)]
vacay_df = vacay_df[(vacay_df["Wind Speed"]) <= 7]
vacay_df = vacay_df[(vacay_df["Humidity"] > 30) & (vacay_df["Humidity"] <=60)]
vacay_df = vacay_df[(vacay_df["Cloudiness"]) <= 20]

#Drop null values and print dataframe
vacay_df.dropna()
vacay_df
 

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
62,62,Hurghada,0,EG,1584285006,39,27.26,33.81,78.80,4.70
91,91,Castro,1,BR,1584285110,57,-24.79,-50.01,80.91,6.38
365,365,Santa Luzia,20,BR,1584285418,54,-19.77,-43.85,82.40,4.70
489,489,Esmeraldas,20,BR,1584285557,51,-19.76,-44.31,82.40,2.24
517,517,Ratnagiri,0,IN,1584285588,58,16.98,73.30,74.07,2.93
554,554,Santiago,0,CL,1584285609,38,-33.46,-70.65,77.00,6.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Define hotel_df and add Hotel Name as a column to the dataframe
hotel_df = vacay_df
hotel_df.insert(9, "Hotel Name", " ")
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
62,Hurghada,0,EG,1584285006,39,27.26,33.81,78.80,4.70,
91,Castro,1,BR,1584285110,57,-24.79,-50.01,80.91,6.38,
365,Santa Luzia,20,BR,1584285418,54,-19.77,-43.85,82.40,4.70,
489,Esmeraldas,20,BR,1584285557,51,-19.76,-44.31,82.40,2.24,
517,Ratnagiri,0,IN,1584285588,58,16.98,73.30,74.07,2.93,
554,Santiago,0,CL,1584285609,38,-33.46,-70.65,77.00,6.17,


In [11]:
#Define variables for Google Place API parameters
target_radius = 5000
target_search_type = "lodging"

#Set up Parameters for Google Place API
params = {
"radius": target_radius,
    "types": target_search_type,
    "key": g_key
}

#Set up loop to iterate through each row in hotel_df 
for index, row in hotel_df.iterrows():
    # Retrieve lat and lng values from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # Update location by using lat and lng values
    params["location"] = f"{lat},{lng}"
    
    #Define url to be used to retrieve hotel details
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #Make API call to retrieve hotel details.
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    results = response['results']
    
    #Print hotel details for each vacation city location 
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    #Skip if there is an error encountered
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        





Retrieving Results for Index 62: Hurghada.
Closest hotel is Sea Star Beau Rivage Hotel.
Retrieving Results for Index 91: Castro.
Closest hotel is CHACARA BAILLY.
Retrieving Results for Index 365: Santa Luzia.
Closest hotel is Pousada Chao Earth.
Retrieving Results for Index 489: Esmeraldas.
Closest hotel is Hotel Fazenda Lagoa Azul.
Retrieving Results for Index 517: Ratnagiri.
Closest hotel is Hotel Landmark.
Retrieving Results for Index 554: Santiago.
Closest hotel is Hotel Panamericano.


In [12]:

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
62,Hurghada,0,EG,1584285006,39,27.26,33.81,78.80,4.70,Sea Star Beau Rivage Hotel
91,Castro,1,BR,1584285110,57,-24.79,-50.01,80.91,6.38,CHACARA BAILLY
365,Santa Luzia,20,BR,1584285418,54,-19.77,-43.85,82.40,4.70,Pousada Chao Earth
489,Esmeraldas,20,BR,1584285557,51,-19.76,-44.31,82.40,2.24,Hotel Fazenda Lagoa Azul
517,Ratnagiri,0,IN,1584285588,58,16.98,73.30,74.07,2.93,Hotel Landmark
554,Santiago,0,CL,1584285609,38,-33.46,-70.65,77.00,6.17,Hotel Panamericano


In [13]:
narrowed_city_df = hotel_df

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

#Display Map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig



Figure(layout=FigureLayout(height='420px'))